<a href="https://colab.research.google.com/github/mkaramib/NLP/blob/main/NER/NER_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Named Entity Recognition(NER)
In this Jupyter notebook, a NER using LSTM will be implemented. I will use Trax as the development library. 


In [ ]:
import os


## Data
This section loads the data such as sentences, tags, words, etc. 

In [ ]:
# define corresponding files.
sentences_file = "sentence.txt"
labels_file = "labels.txt"
word_file = "words.txt"
tag_file = "tags.txt"

### Sentences and Labels
Sentences and corresponding sequence of NER labels are loaded. 

In [ ]:
def load_sentences(file):
  